<a href="https://colab.research.google.com/github/paulodowd/EMATM0054_53/blob/main/L4_SystemIntegration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 4: System Integration


This labsheet will help you to bring the components of your solution together, and to think about the higher-level tasks that your 3Pi+ robot must be programmed to complete.

<hr><br><br><br><br>


# Finite State Machine (FSM)

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/theory.png" align="left"> The below illustration is a state diagram for a solution to Assessment 1.  It is regarded as a finite state machine because it should capture all possible states and transitions that the system will operate.Whilst you may not have all parts of the problem solved yet, it is good to look ahead and think about the structure of your code.  



<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/fsm.png">
</p>

When we review the above diagram, we can regard each state as a `robot behaviour`.  In particular, it is important to note:
- Most states loop back into themselves.  For example, the `calibration` transitions back into `calibration` whilst the robot has not completed `2 rotations`.  `return to start area` loops back into `return to start area` whilst the robot has not reached the start area.
  - This principle should be reflected in the functions you have been recommended to write that are **non-blocking**.  Therefore, whilst your code is within a particular state, it should be able to repeatedly call the same **non-blocking** function.
  - Each state (or behaviour) is therefore checking for conditions to transition to other states.
  - You can write your code so that this **state transition** is either managed by each state itself, or a section of code at the beginning of `loop()` can determine the correct state on each iteration.
- Some states represented may in fact be more complicated than shown.  In fact, a state like `search area` may itself have a state machine.  
  - For example, in the Assessment 1 Video Submission example, the 3Pi+ robot can be observed to stay within the search area marked by a grid.  Therefore, within the state `search area`, there is a detection of the robot position operating to keep the robot on the right hand portion of the coursework map.

We can therefore quickly illustrate a general code structure that would enable for your code to be organised as a finite state machine:

```c

#define STATE_CALIBRATE   0  // give each behaviour a number
#define STATE_LEAVE_START 1
#define STATE_SEARCH_AREA 2
int state;                  // this will track which state is active

void setup() {

  // Which state should the robot start in?
  state = STATE_CALIBRATE;

}

void loop() {

  // Update FSM.  Check which state we are currently
  // in.  States will transition themselves.
  if( state == STATE_CALIBRATE ) {

    // call calibration routine (non-blocking)
    doCalibration();

    // Should we move to the next state?
    if( ???? >  ???? ) {

      // move the state variable to next state
      state = STATE_LEAVE_START;

      // Prepare any variables or functions
      setForwards( 2000 );

    }


  } else if( state == STATE_LEAVE_START ) {

    // Finished moving forwards? (non-blocking)
    bool status = checkForwards();
    
    // Should we move to next state?
    if( status == false ) {
      
      // move the state variable to next state
      state = STATE_SEARCH_AREA;

      // Prepare any functions or variables
      // ...
    }


  } else if( state == STATE_SEARCH_AREA ) {

      // continue search (non-blocking)
      checkSearch();

      // Should we move to next state?
      if( ???? > ???? ) {

        // move the state variable to next state
        state = ????;

      } else if ( ???? > ???? ) {

        // You can also branch which state is next
        state = ????;
        
      }

  } // End of FSM update

} // End of loop()


```

## Exercise 1: Build a Finite State Machine (FSM)
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/exercise.png" align="left">  **Exercise:** Take the time to refactor your code to fit into a FSM framework, such as the extract above.  


<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/tick.png" align="left"> Organising your code into a FSM has several advantages.  The most important concerns debugging.  With a FSM, you can enable and disable parts of your code by simply changing the state transitions, or commenting out state transitions, or even over-riding the state at the top of `loop()` (for example, by setting `state = STATE_DEBUG;` as the first thing in `loop()`, the `if()` statement would always select the code for `state == STATE_DEBUG`).  

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/tick.png" align="left"> When you are working towards the final solution for Assessment 1, your solution will have grown in size and complexity.  If it takes 3 minutes to watch your robot operate, you could waste a lot of time.  For example, if you are debugging **return to start area**, it is not strictly necessary to observe the whole solution.  Instead, you can set the robot to begin in the state to "return to start", and start the kinematics from any position to test (e.g. `pose.initialise( 100, -300, 0);`).  This way, you avoid having to watch the robot slowly search for a magnet, etc.  With a FSM, once you have finished debugging, you can easily return your code back to full functionality.

<hr><br><br><br><br>

# High Level Movement Routines (Kinematics)

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/theory.png" align="left">  To complete Assessment 1 fully, your robot is required to make two movements that require information on the position of the robot with respect to where it started.  These are:


- Returning to the start area.
- Returning to the magnet location.

In the following exercises, we will first ensure that the robot can estimate it's pose, and then adapt previous movement routines to utilise pose information.

The <a href="https://github.com/paulodowd/EMATM0054_53/tree/main/3Pi_CodeStub">code stub</a> that has been provided to you has a complete class for kinematics (`Kinematics.h`).  The kinematics is the mathematics to estimate the robot location based on the number of encoder counts that have occured.  This technique is known as **dead reckoning**, a form of **odometry**.  

**Odometry** is a more general term and means to use sensors to estimate a change of position over time.  This could include sensors such as gyroscopes, accelerometers, etc - which are not covered in these labsheets.  

**Dead-reckoning** is a more specific term, which informally means to attempt to "keep count" relative to a previously known or fixed position.  Our robot has a known starting position - the location where it is activated (powered on), which we can consider to mean $x_{I}=0$, $Y_{I}=0$ and $\theta_{I}=0$, where the subscript `I` means "global frame".  We can understand "global frame" to mean, a position relative to where the robot started.  

To do this, you need to ensure that your code for your 3Pi+ robot has the following essential lines of code (with the rest removed here):

```c
#include "Encoders.h"   // provide automatic counting of encoders
#include "Kinematics.h" // maths for odometry/dead-reckoning

// Create a global instance of the kinematics class
Kinematics_c pose;

void setup() {

  // Initialises the routines that will automatically
  // count the rotations of the wheels.
  setupEncoder0();
  setupEncoder1();

  // Initialise the pose of the robot to x=0, y=0,
  // theta=0.  
  pose.initialise( 0, 0, 0 );

}

void loop() {

  // Instruct the kinematics to perform an update
  // of the robot position
  // It is recommended you schedule this to occur at
  // a fixed interval, such as 20ms.
  pose.update();
}
```



## Exercise 2: Calibrate the Kinematics

The kinematics provided to you has the measurements provided by the manufacturer as the model parameters.  These parameters can be improved by conducting a manual calibration.  Unlike previous calibration exercises, this cailbration is not expected to be automatic or autonomous.  Instead, we will need to produce some simple movements with the robot and inspect the outcomes.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/exercise.png" align="left">  **Exercise:** We will test that the `Kinematics_c` class is working correctly, and then follow some steps to calibrate the parameters of the kinematics.  Incorporate the above code extract into code that operates your 3Pi+ robot.  You can either start a new sketch to keep things simple (recommended), or attempt to integrate the above essential lines of code into your current program.  If you are using an existing program, you will want to disable all behaviours/operations whilst we work to calibrate the kinematics (e.g., we do not want the robot to try to move).



1. Ensure that `pose.update();` is being called within `loop()`. It is highly recommended that you schedule this function call using `millis()` to operate at an interval such as 20ms.  This technique was originally covered in Labsheet 0.

2. Test that your program is updating values within the kinematics class by using Serial.print() on the variables `pose.x`, `pose.y`, `pose.theta`.  
  - Add these print statments into `loop()`.
  - Remember that you can increase the precision of `Serial.print()` by adding a whole number as a second argument, e.g. `Serial.print( pose.x, 4 );`



3. **Calibrating Translation**: The coursework map has a grid with 10mm spacing between cells drawn in light grey.  Use this to calibrate the **x contribution** (forward translation) of your robot.  
  - Position your robot with the axles of both wheels aligned over the line marked 0mm.  
  - **Without motor power**, push the robot along in a straight line, stopping at the line marked 100mm.
  - Inspect the output for the **x coordinate** from the pose instance of `Kinematics_c`.  You can do this simply with `Serial.print( pose.x, 4 );`.
  - If your x coordinate is more than 100mm, you will need to reduce the parameter `wheel_radius` in the file `Kinematics.h`.
  - If your x coordinate is less than 100mm, you will need to increase the parameter `wheel_radius` in the file `Kinematics.h`.
  - Repeat this process improving the reported value of X.

<p align="center">
<img width="250px" src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/3Pi_grid_start.jpg"> &nbsp; <img width="250px" src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/3Pi_grid_end.jpg"><br>
<small>The robot positioned at the start of the grid, with the wheel axle positioned over the line 0mm (left).  The robot at the end of the grid, with the wheel axle positioned over the line marked 100mm (right).
</p>

4. **Calibrating Rotation:** The coursework map has a radial figure of angles in radians.  Use this to calibrate the rotation (**theta contribution**) of your robot.
  - Position the robot exactly within the centre of the radial figure.  You can do this by lining up the wheel axles with a horizontal line, and by lining up the gap between the bumper levers with the vertical line.  
  - **Without motor power** rotate the robot to a pre-determined angle, such as 90 degrees ($\frac{\pi}{2}$).
  - Inspect the reported value of `theta`.  
  - Increase or decrease the parameter `wheel_sep` (wheel seperation) within `Kinematics.h` depending on whether the reported value is too high or too low.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/hypothesis.png" align="left">**Hypothesise:** Following these instructions, we have calibrated translation before rotation.  Why might the order of calibrating these be important?

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/hypothesis.png" align="left">**Hypothesise:** The <a href="https://www.pololu.com/product/4975/resources">manufacturer documentation</a> for the Pololu 3Pi+ is quite detailed and includes important measurements for the robot.  Why might the reported measurements be inaccurate? What factors may influence the effect of these parameters on the kinematic model?

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left">  The kinematic model provided in `Kinematics.h` is a simpler form which assumes that the robot will either translate or rotate (exclusively).  The kinematics model movement as a translation, followed by a rotation.  Therefore, the model will represent curved motion as a straight-line approximation.  Some error is inherent to the model.  However, the simple kinematic model has proven sufficient to complete Assessment 1 to a high standard.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/hypothesis.png" align="left">**Hypothesise:** It would make sense that calling `pose.update()` as quickly as possible would reduce the error of a straight-line approximation of a curve (the curve would be represented by smaller segments, a "higher resolution").  However, this does not seem to be the case in reality.  Why might this be?

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left">  If you would like more information on the kinematic model, a good source of further reading is the book: **Siegwart, R., Nourbakhsh, I.R., Scaramuzza, D., Arkin, R.C. (2011) Introduction to Autonomous Mobile Robots 2nd Edition.  MIT Press. ISBN-10 0262015358.** (<a href="https://bris.on.worldcat.org/oclc/708414202">available in the University of Bristol Library</a>)

<hr><br><br><br><br>


## Exercise 3: Turning a Precise Angle

In previous labsheets, you have been guided towards writing a **non-blocking** function to perform forwards travel, turning, or both - with a duration specified in milliseconds (_ms_).  You may have found values of PWM and duration to create a turn of 90 degrees, for example.  A clear issue with this approach is that it is **open-loop**, meaning that the robot does not know if it has achieved the desired turn, and the performance may vary between robots, or within the robot's operational lifetime.  This can lead to a lot of frustration, where we feel that the robot "should" turn 90 degrees - but of course there is no reason it "should".

With the kinematic model, we can now instruct the robot to perform a turn to face a desired angle.  This means duration is no longer required, the robot can manage the turn operation until the angle is achieved - **closed-loop control**.





<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/3Pi_DiffAngles.png">
</p>

In the above illustrations we see 3 different rotations of the 3Pi+ robot with respect to the global coordinate frame ($X_{I}$, $Y_{I}$).  The blue arrow represents the robots current rotation, $\theta_{I}$ (`pose.theta`).  The green arrow represents a desired target angle.  

In the first panel, the problem appears easy.  $\theta_{I}$ is zero, and the robot will rotate positively towards the green arrow.  If the green arrow was $45°$, then $45 > 0$ (i.e. target > $\theta_{I}$), and we could take this to mean rotate left.

In the second panel, we can imagine that $\theta_{I}$ is now 350°, and the green arrow (target angle) is 10°.  Now, 10° < 350° (i.e. target < $\theta_{I}$, **less than!**), and so the robot would rotate **right** - the long way around.

Therefore, when we consider this problem, we can articulate it as **requiring the smallest angle between two angles**.  Let's call "the smallest angle between two angles" $\theta_{s}$.  To produce an effective turning behaviour, we therefore want to:
- work out the current value of $\theta_{s}$, for the current rotation $\theta_{I}$ and the desired angle.
- rotate in the right direction to make the next calculation of $\theta_{s}$ smaller (minimisation problem)
- stop moving once $\theta_{s} = 0 $ (the target has been reached).

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/exercise.png" align="left">  **Exercise:** Using the **non-blocking** functions you have previously written for your robot movement, update them so that `setTurn()` (or whatever you called it) now takes a `float` argument for the `target_angle`.  Update your `checkTurn()` function (or whatever you called it) to now check if $\theta_{s}$ has reached a near zero value.  If it has, the robot should stop.  If it has not, the robot should continue turning in the correct direction.
- Add a global variable that represents `target_angle`.  Whilst we could change this value directly from anywhere in the code, this could lead to many debugging problems.  Instead, use the function `setTurn()` to set the value. This way, the turn operation will be standardised and consistent.  You will want to call `setTurn()` once, when the turn operation is first required. After this, you will want to call `checkTurn()` to complete the turn in a non-blocking way.
- It is recommended you only update your kinematics at one location in your code, to keep the execution of your code as predictable and as consistent as possible.  A good location is therefore within `loop()`, because we will want to update the kinematics on every iteration of `loop()`.  Therefore, it is not necessary to call `pose.update()` within your turning functions.
- Remember that the kinematics class (`Kinematics_c`) instance `pose` is defined as a global variable.  Therefore, your functions `setTurn()` and `checkTurn()` can access the instance `pose.` easily.
- The issue of two angles crossing the 0/360 axis of a circle is a well understood problem.  The <a href="https://en.wikipedia.org/wiki/Atan2">atan2</a> function helps significantly in solving this problem.
- Stopping precisely can be difficult.  Once you have a calculation for $\theta_{s}$, use an `if()` statement to check if this has reached a small enough value.  For example:

```c
// You can define a value at the top of your program
// so that you don't have to search for it when you
// want to change it.
#define TURN_THRESHOLD 0.1

bool checkTurn() {

  // Is the difference between current rotation and
  // target_angle now sufficiently small?
  // Function abs() returns the unsigned value.
  if( abs(angle_difference) < TURN_THRESHOLD ) {

    // Stop robot
    motors.setPWM( 0, 0 );

    // Signal turn complete.
    return true;

  }

  // Signal turn has not finished.
  return false;

} // End of checkTurn()

```

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/validate.png" align="left"> **Validate:** validate that your turning operation is working by trying various test cases.  It is likely that you cannot know in advance what rotation your robot will have when it must complete a turn towards a target angle.  Remember that you can be more efficient in your debugging by isolating parts of your code.  For example, it will cost a lot of time if you have to watch your robot peform line sensor and magnetometer calibrations every time you activate it.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left"> If you have completed the Advanced Exercises for Labsheet 1, you can improve this turn operation further.  It is possible to implement a PI-Controller that takes $\theta_{s}$ (the smallest difference between two angles) and produces as output a speed demand (in encoder counts per ms, or mm/ms).  Remember that a PID is really mapping an input to an output, and it is essentially unit-agnostic.  This speed demand output is then feed as an input into the PI-Controllers for wheel speeds.  This is referred to as **nested PID control**, where one PID controller is manipulating another PID controller.  The advantage of this is that your robot will be able to move very slowly, and close very small gaps in rotation to face a precise angle.

<hr><br><br><br><br>

## Exercise 4: Travelling to X,Y Coordinates.

In the previous exercise you completed code to allow your robot to rotate to face a specified angle.  This is a key component to travelling to a target X and Y location.  We can consider travelling to an X,Y coordinate as having two discrete steps:
- Rotate to face the target.
- Move forwards until the distance is to the target X,Y coordinate is 0.

If these steps are not seperated initially, your robot will produce a very large turn, like so:

<p align="center">
<img width="400px" src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/3Pi_LongTurn.png" >
</p>

However, it is **advantageous** if your robot corrects for error in rotation whilst it travels after the initial rotation.  


<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/stop.png" align="left"> The main issue with the above behaviour is that it is a **requirement of Assessment 1** that your robot takes the **shortest path** back to the start area, and the **shortest path** back to the magnet location.  Therefore, a large looping movement is in breach of this requirement.  Some deviation is acceptable, as long as it is clear that your robot is attempting the shortest path.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left"> **Information:** There are a few other issues with the above movement.  First, it could send your robot outside the coursework map area, which would be in breach of the requirements for Assessment 1.  Second, to minimise error in kinematics, you will also want to be relatively efficient with the types of movement your robot produces.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/exercise.png" align="left">  **Exercise:** For this exercise, program your robot to begin at some non-zero coordinates for $X_{I}$ and $Y_{I}$.  We will focus on getting the robot to travel back to the origin.  You may want to first discover a set of test coordinates somewhere on the coursework map by simply pushing your robot to the location whilst it prints `pose.x` and `pose.y`.

Reviewing the below illustration, there will be **two angles** that we are concerned with:
- The angle from the robot location to the origin, marked $\theta_{H}$ ('H' for home).
- The current rotation of the robot with respect to the global frame, $\theta_{I}$.

This presents two angles, a similar condition to the previous exercise ("Turning a Precise Angle").

<p align="center">
<img width="500px" src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/3Pi_GoingHome.png">
</p>

1. Manipulate $\theta_{H}$ so that we understand the angle required from the robot's point of view.  E.g., looking from the robot back towards the origin.
2. Call your set of turn functions, and test that your robot can rotate to face the origin.  
  - Test this from different positions on the coursework map.

<hr><br><br><br><br>


# Search Routines

The final part of improving your solution may revolve around the choices you make on the search routine to find the magnet.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/hypothesis.png" align="left">**Hypothesise:** If kinematic error is accumulating, what would and when we expect to observe the effect?  How could we measure this?  We know that the kinematics (dead-reckoning) will have some degree of error in the estimation of the robot position.  Furthermore, the quantity of error will be effected by factors such as **(a)** the total time spent moving **(b)** any asymmetry in movement **(c)** external factors, such as wheel slip, **(d)** any substantial `delay()` in your code, **(etc)**.  Therefore, it is worthwhile attempting to observe, theorise and predict on an optimal search routine to minimise some of those factors.  

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/Search_Good_Bad.png">
</p>

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/observation.png" align="left"> **Observations:** Regarding the above, the first observation we can make is that only half of the coursework map will be the area of interest.  This is because the magnet will be placed into the green area (righthand side).  Therefore, we can improve the overall chance of success, and potentially the time spent travelling, by only searching the green (righthand) area.  Therefore, it is perfectly reasonable that your robot would first of all move to the location of the search area grid.

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/Search_Boustrophedon.png">
</p>

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/observation.png" align="left"> **Observations:** Regarding the above, a second obvious solution is to program a sequence of straight-line travel and 90° turns - known as a Boustrophedon path.  However, this path has a lot of unncessary travel and rotation.  There will also be an issue of how close each pass of the path should be.

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/Search_LineFollow.png">
</p>

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/observation.png" align="left"> **Observations:** Regarding the above, it is possible to program the robot to follow the perimeter of the coursework map.  This would make it more probably to find the magnet when it is positioned at the edge of the search area grid. However, it would mean it would be more unlikely to detect the magnet in the central region.

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/Search_Ballistic.png">
</p>

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/observation.png" align="left"> **Observations:** Regarding the above, it is also possible to program the robot to simply move forwards and then turn away from the bounding box of the coursework map.  This would likely cause the robot to "bounce" around the coursework map, and it would be less likely that the robot would detect the magnet in the corner areas.

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/Search_Efficiency.png">
</p>

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/observation.png" align="left"> **Observations:** Regarding the above, it makes sense that if your robot is about to leave the search grid area, it should turn around and go back to the search area.  This would improve the efficiency of the search overall.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/hypothesis.png" align="left">**Hypothesise:** The above discussion is not exhaustive.  You are encouraged to develop your own novel solutions to this problem.

<hr><br><br><br><br>
